# Función de costo dentro de la arquitectura

## Ejemplo mas simple del uso de `pytorch_lightning`

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pytorch_lightning as pl

# Preparación de los datos
X_train = torch.randn(1000, 10)
y_train = torch.randn(1000, 1)
X_val = torch.randn(200, 10)
y_val = torch.randn(200, 1)
X_test = torch.randn(200, 10)
y_test = torch.randn(200, 1)

# Definición del datamodule
class SimpleDataModule(pl.LightningDataModule):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test, batch_size=32):
        super().__init__()
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.batch_size = batch_size

    def train_dataloader(self):
        train_dataset = TensorDataset(self.X_train, self.y_train)
        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        val_dataset = TensorDataset(self.X_val, self.y_val)
        return DataLoader(val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        test_dataset = TensorDataset(self.X_test, self.y_test)
        return DataLoader(test_dataset, batch_size=self.batch_size)

# Definición del modelo
class SimpleModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Linear(10, 1)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = torch.nn.functional.mse_loss(output, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = torch.nn.functional.mse_loss(output, y)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        output = self(x)
        loss = torch.nn.functional.mse_loss(output, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# Entrenamiento del modelo
datamodule = SimpleDataModule(X_train, y_train, X_val, y_val, X_test, y_test)
model = SimpleModel()
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, datamodule)

## Resumen del Funcionamiento del Código

El código proporcionado define una clase llamada `LNNP` que sirve como un contenedor para un modelo de redes neuronales potenciales (NNP, por sus siglas en inglés) implementado en TorchMD-Net utilizando PyTorch Lightning. A continuación se describe el funcionamiento general del código:

- **Importaciones**: Se importan las bibliotecas necesarias, incluyendo PyTorch, PyTorch Lightning y módulos específicos de TorchMD-Net.

- **Clase `LNNP`**: Se define la clase `LNNP`, que hereda de `LightningModule`. Esta clase encapsula la lógica del modelo NNP y proporciona métodos para entrenamiento, validación y prueba.

- **Inicialización**: En el método `__init__`, se configuran los hiperparámetros, se crea o carga el modelo NNP, se inicializan variables para suavizado exponencial y almacenamiento de pérdidas.

- **Configuración de Optimizadores**: En el método `configure_optimizers`, se configuran el optimizador AdamW y el planificador de tasa de aprendizaje ReduceLROnPlateau.

- **Paso Forward**: En el método `forward`, se realiza el paso hacia adelante del modelo NNP para generar predicciones a partir de datos de entrada.

- **Paso de Entrenamiento**: En el método `training_step`, se realiza un paso de entrenamiento utilizando la función de pérdida MSE.

- **Paso de Validación y Prueba**: En los métodos `validation_step` y `test_step`, se realizan pasos de validación y prueba respectivamente, utilizando la función de pérdida L1.

- **Cálculo de Pérdidas y Actualización de Parámetros**: En el método `step`, se calculan las pérdidas y se actualizan los parámetros del modelo.

- **Actualización del Optimizador**: En el método `optimizer_step`, se realiza un paso de optimización, incluyendo el escalado de la tasa de aprendizaje durante el calentamiento.

- **Registro de Métricas**: En los métodos `training_epoch_end` y `validation_epoch_end`, se registran métricas como pérdidas y tasas de aprendizaje al final de cada época de entrenamiento y validación.

- **Reinicio de Pérdidas y EMA**: En los métodos `_reset_losses_dict` y `_reset_ema_dict`, se reinician las variables de pérdidas y suavizado exponencial móvil al comienzo de cada época.

Este resumen proporciona una visión general del funcionamiento del código, destacando los principales componentes y procesos involucrados en el entrenamiento, validación y prueba del modelo NNP utilizando PyTorch Lightning.


## Implementación dentro de la arquitectura

## Importaciones

Importamos las bibliotecas necesarias para nuestro módulo. Esto incluye:

- `torch`: La biblioteca principal de PyTorch para cálculos de tensor y operaciones en GPU.
- `AdamW`: Un optimizador que implementa el algoritmo Adam con corrección de peso.
- `ReduceLROnPlateau`: Un programador de tasa de aprendizaje que reduce la tasa de aprendizaje cuando una métrica ha dejado de mejorar.
- `mse_loss`, `l1_loss`: Funciones de pérdida Mean Squared Error (MSE) y Mean Absolute Error (L1).
- `Tensor`: Tipo de datos de tensor de PyTorch.
- `Optional`, `Dict`, `Tuple`: Tipos de datos de Python utilizados para definir tipos de argumentos y retornos de funciones.
- `LightningModule`: Clase base proporcionada por PyTorch Lightning para la creación de módulos de red neuronal.
- `create_model`, `load_model`: Funciones de un módulo llamado `torchmdnet` para crear y cargar modelos.

Estas importaciones son esenciales para las funcionalidades posteriores del módulo.


In [ ]:
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import mse_loss, l1_loss
from torch import Tensor
from typing import Optional, Dict, Tuple

from pytorch_lightning import LightningModule
from torchmdnet.models.model import create_model, load_model

## Definición de la clase `LNNP`

Creamos una clase llamada `LNNP` que hereda de `LightningModule`. Esta clase sirve como un contenedor para nuestra red neuronal potencial (NNP, por sus siglas en inglés) implementada en TorchMD-Net. La documentación de la clase es la siguiente:

- **Nombre**: `LNNP`
- **Hereda de**: `LightningModule`
- **Descripción**: Se trata de un contenedor que envuelve las funcionalidades de la red neuronal potencial (NNP) implementada en TorchMD-Net utilizando PyTorch Lightning. Esta clase nos permite entrenar, validar y probar la NNP de manera eficiente con las capacidades adicionales proporcionadas por PyTorch Lightning.


In [ ]:
class LNNP(LightningModule):
    """
    Lightning wrapper for the Neural Network Potentials in TorchMD-Net.
    """

## Constructor `__init__`

Definimos el constructor `__init__` de la clase `LNNP`. Este método se llama automáticamente cuando se crea una instancia de la clase `LNNP`. Aquí está lo que hace:

- **Argumentos**:
  - `hparams`: Un diccionario que contiene hiperparámetros para configurar el modelo.
  - `prior_model`: Un modelo previo opcional que se puede utilizar para inicializar el modelo.
  - `mean`, `std`: Medias y desviaciones estándar opcionales para normalizar los datos de entrada.
- **Superllamada**: Llamamos al constructor de la clase base (`LightningModule`) usando `super()`, asegurándonos de inicializar todos los atributos y métodos heredados.
- **Ajuste de hiperparámetros**: Añadimos claves faltantes ("charge" y "spin") al diccionario de hiperparámetros `hparams` si no están presentes. Luego, guardamos todos los hiperparámetros utilizando `save_hyperparameters(hparams)`, lo que permite el acceso a los hiperparámetros en todo el módulo con `self.hparams`.
- **Inicialización del modelo**: Si se especifica un modelo para cargar (`load_model` en los hiperparámetros), cargamos ese modelo utilizando la función `load_model`. De lo contrario, creamos un nuevo modelo utilizando la función `create_model`, pasando los hiperparámetros y cualquier modelo previo, media y desviación estándar si están disponibles.
- **Inicialización de la suavización exponencial**: Inicializamos una variable `ema` (Exponential Moving Average) y reiniciamos un diccionario asociado utilizando `_reset_ema_dict()`.
- **Inicialización de la colección de pérdidas**: Inicializamos una variable `losses` y reiniciamos un diccionario asociado utilizando `_reset_losses_dict()`.


In [ ]:
    def __init__(self, hparams, prior_model=None, mean=None, std=None):
        super(LNNP, self).__init__()

        if "charge" not in hparams:
            hparams["charge"] = False
        if "spin" not in hparams:
            hparams["spin"] = False

        self.save_hyperparameters(hparams)

        if self.hparams.load_model:
            self.model = load_model(self.hparams.load_model, args=self.hparams)
        else:
            self.model = create_model(self.hparams, prior_model, mean, std)

        # initialize exponential smoothing
        self.ema = None
        self._reset_ema_dict()

        # initialize loss collection
        self.losses = None
        self._reset_losses_dict()

## Configuración de los optimizadores

Definimos el método `configure_optimizers`, que se encarga de configurar los optimizadores y los planificadores de tasas de aprendizaje para el entrenamiento. Aquí está lo que hace:

- **Optimizador**: Utilizamos el optimizador AdamW para optimizar los parámetros del modelo. Configuramos el optimizador con los siguientes argumentos:
  - `parameters()`: Los parámetros del modelo que se optimizarán.
  - `lr`: Tasa de aprendizaje, tomada de los hiperparámetros (`self.hparams.lr`).
  - `weight_decay`: Término de regularización, tomado de los hiperparámetros (`self.hparams.weight_decay`).
- **Planificador de tasas de aprendizaje**: Utilizamos `ReduceLROnPlateau` para ajustar la tasa de aprendizaje dinámicamente durante el entrenamiento. Configuramos el planificador con los siguientes argumentos:
  - `optimizer`: El optimizador que estamos utilizando.
  - `mode`: El modo de reducción de la tasa de aprendizaje, en este caso, "min" para reducir la tasa de aprendizaje cuando la métrica monitoreada deja de disminuir.
  - `factor`, `patience`, `min_lr`: Factores de reducción, paciencia antes de reducir y el mínimo de la tasa de aprendizaje permitido, tomados de los hiperparámetros (`self.hparams.lr_factor`, `self.hparams.lr_patience`, `self.hparams.lr_min`).
- **Configuración del planificador de tasas de aprendizaje**: Creamos un diccionario `lr_scheduler` que contiene la configuración del planificador de tasas de aprendizaje. Esto incluye el planificador mismo, la métrica que se está monitoreando, el intervalo en el que se aplicará el planificador y la frecuencia con la que se llamará, todo tomado de los hiperparámetros.
- **Retorno**: Devolvemos una lista de optimizadores y una lista de planificadores de tasas de aprendizaje para ser utilizados durante el entrenamiento.


In [ ]:
    def configure_optimizers(self):
        optimizer = AdamW(
            self.model.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay,
        )
        scheduler = ReduceLROnPlateau(
            optimizer,
            "min",
            factor=self.hparams.lr_factor,
            patience=self.hparams.lr_patience,
            min_lr=self.hparams.lr_min,
        )
        lr_scheduler = {
            "scheduler": scheduler,
            "monitor": getattr(self.hparams, "lr_metric", "val_loss"),
            "interval": "epoch",
            "frequency": 1,
        }
        return [optimizer], [lr_scheduler]

## Método `forward`

Definimos el método `forward`, que se encarga de pasar los datos a través del modelo. Aquí está lo que hace:

- **Argumentos**:
  - `z`: Tensor de características de entrada.
  - `pos`: Tensor de posiciones de los átomos.
  - `batch`: Tensor opcional que indica la pertenencia de cada átomo a un batch específico.
  - `q`: Tensor opcional de cargas de átomos.
  - `s`: Tensor opcional de spin de los átomos.
  - `extra_args`: Argumentos adicionales opcionales.
- **Retorno**:
  - Una tupla que contiene:
    - Tensor de salida del modelo.
    - Tensor opcional adicional (puede ser `None`).

Dentro del método, llamamos al método `forward` del modelo subyacente (`self.model`) pasando los argumentos proporcionados. Esto permite que los datos fluyan a través del modelo y obtengamos la salida correspondiente.


In [ ]:
    def forward(self,
                z: Tensor,
                pos: Tensor,
                batch: Optional[Tensor] = None,
                q: Optional[Tensor] = None,
                s: Optional[Tensor] = None,
                extra_args: Optional[Dict[str, Tensor]] = None
                ) -> Tuple[Tensor, Optional[Tensor]]:

        return self.model(z, pos, batch=batch, q=q, s=s, extra_args=extra_args)

## Método `training_step`

Definimos el método `training_step`, que se encarga de realizar un paso de entrenamiento durante el entrenamiento del modelo. Aquí está lo que hace:

- **Argumentos**:
  - `batch`: Un lote de datos de entrenamiento.
  - `batch_idx`: El índice del lote actual.
- **Retorno**:
  - El resultado del método `step`, que contiene la pérdida calculada durante el paso de entrenamiento.

Dentro del método, llamamos a un método interno llamado `step`, pasando el lote de datos de entrenamiento, la función de pérdida MSE (`mse_loss`) y la cadena "train" para indicar que estamos en la fase de entrenamiento. El método `step` se encarga de realizar el cálculo de la pérdida y cualquier otro procesamiento necesario para un paso de entrenamiento.


In [ ]:
    def training_step(self, batch, batch_idx):
        return self.step(batch, mse_loss, "train")

## Método `validation_step`

Definimos el método `validation_step`, que se encarga de realizar un paso de validación durante el entrenamiento del modelo. Aquí está lo que hace:

- **Argumentos**:
  - `batch`: Un lote de datos de validación.
  - `batch_idx`: El índice del lote actual.
  - `*args`: Argumentos adicionales, se utiliza para determinar si estamos en la fase de prueba o validación.
- **Retorno**:
  - El resultado del método `step`, que contiene la pérdida calculada durante el paso de validación o prueba.

Dentro del método, verificamos la longitud de los argumentos y su valor para determinar si estamos en la fase de validación o prueba. Si no se proporcionan argumentos adicionales o si el primer argumento es cero, realizamos un paso de validación utilizando la función de pérdida MSE (`mse_loss`). Si el primer argumento es diferente de cero, realizamos un paso de prueba utilizando la función de pérdida L1 (`l1_loss`). En ambos casos, llamamos al método `step` para realizar el cálculo de la pérdida y cualquier otro procesamiento necesario.


In [ ]:
    def validation_step(self, batch, batch_idx, *args):
        if len(args) == 0 or (len(args) > 0 and args[0] == 0):
            # validation step
            return self.step(batch, mse_loss, "val")
        # test step
        return self.step(batch, l1_loss, "test")

## Método `test_step`

Definimos el método `test_step`, que se encarga de realizar un paso de prueba durante la evaluación del modelo. Aquí está lo que hace:

- **Argumentos**:
  - `batch`: Un lote de datos de prueba.
  - `batch_idx`: El índice del lote actual.
- **Retorno**:
  - El resultado del método `step`, que contiene la pérdida calculada durante el paso de prueba.

Dentro del método, llamamos al método `step`, pasando el lote de datos de prueba, la función de pérdida L1 (`l1_loss`) y la cadena "test" para indicar que estamos en la fase de prueba. El método `step` se encarga de realizar el cálculo de la pérdida y cualquier otro procesamiento necesario para un paso de prueba.


In [ ]:
    def test_step(self, batch, batch_idx):
        return self.step(batch, l1_loss, "test")

## Método `step`

Definimos el método `step`, que encapsula el procesamiento común para un paso de entrenamiento, validación o prueba. Aquí está lo que hace:

- **Argumentos**:
  - `batch`: Un lote de datos.
  - `loss_fn`: La función de pérdida a utilizar para calcular la pérdida.
  - `stage`: Una cadena que indica la etapa actual, que puede ser "train" (entrenamiento), "val" (validación) o "test" (prueba).
- **Retorno**:
  - La pérdida calculada para el lote actual.

Dentro del método, realizamos los siguientes pasos:

1. **Gestión de los gradientes**: Configuramos el entorno de gradiente según la etapa actual (`train`) o si se requiere el cálculo del gradiente (`derivative` en los hiperparámetros).
2. **Obtención de la salida del modelo**: Llamamos al método `forward` del modelo para obtener las predicciones (`y`) y las negativas de las derivadas (`neg_dy`) utilizando los datos del lote.
3. **Cálculo de las pérdidas**: Calculamos las pérdidas para las predicciones (`y`) y las negativas de las derivadas (`neg_dy`) si están disponibles en el lote de datos. Si se especifica, aplicamos suavizado exponencial a las pérdidas durante el entrenamiento y la validación. Luego, calculamos la pérdida total como la suma ponderada de las pérdidas de las predicciones y las negativas de las derivadas.
4. **Almacenamiento de las pérdidas**: Almacenamos las pérdidas calculadas en la lista de pérdidas correspondiente a la etapa actual (`stage`).
5. **Retorno de la pérdida**: Devolvemos la pérdida total calculada.

Este método encapsula el cálculo de la pérdida y el procesamiento asociado para un paso de entrenamiento, validación o prueba.


In [ ]:
    def step(self, batch, loss_fn, stage):
        with torch.set_grad_enabled(stage == "train" or self.hparams.derivative):
            extra_args = batch.to_dict()
            for a in ('y', 'neg_dy', 'z', 'pos', 'batch', 'q', 's'):
                if a in extra_args:
                    del extra_args[a]
            # TODO: the model doesn't necessarily need to return a derivative once
            # Union typing works under TorchScript (https://github.com/pytorch/pytorch/pull/53180)
            y, neg_dy = self(
                batch.z,
                batch.pos,
                batch=batch.batch,
                q=batch.q if self.hparams.charge else None,
                s=batch.s if self.hparams.spin else None,
                extra_args=extra_args
            )

        loss_y, loss_neg_dy = 0, 0
        if self.hparams.derivative:
            if "y" not in batch:
                # "use" both outputs of the model's forward function but discard the first
                # to only use the negative derivative and avoid 'Expected to have finished reduction
                # in the prior iteration before starting a new one.', which otherwise get's
                # thrown because of setting 'find_unused_parameters=False' in the DDPPlugin
                neg_dy = neg_dy + y.sum() * 0

            # negative derivative loss
            loss_neg_dy = loss_fn(neg_dy, batch.neg_dy)

            if stage in ["train", "val"] and self.hparams.ema_alpha_neg_dy < 1:
                if self.ema[stage + "_neg_dy"] is None:
                    self.ema[stage + "_neg_dy"] = loss_neg_dy.detach()
                # apply exponential smoothing over batches to neg_dy
                loss_neg_dy = (
                    self.hparams.ema_alpha_neg_dy * loss_neg_dy
                    + (1 - self.hparams.ema_alpha_neg_dy) * self.ema[stage + "_neg_dy"]
                )
                self.ema[stage + "_neg_dy"] = loss_neg_dy.detach()

            if self.hparams.neg_dy_weight > 0:
                self.losses[stage + "_neg_dy"].append(loss_neg_dy.detach())

        if "y" in batch:
            if batch.y.ndim == 1:
                batch.y = batch.y.unsqueeze(1)

            # y loss
            loss_y = loss_fn(y, batch.y)

            if stage in ["train", "val"] and self.hparams.ema_alpha_y < 1:
                if self.ema[stage + "_y"] is None:
                    self.ema[stage + "_y"] = loss_y.detach()
                # apply exponential smoothing over batches to y
                loss_y = (
                    self.hparams.ema_alpha_y * loss_y
                    + (1 - self.hparams.ema_alpha_y) * self.ema[stage + "_y"]
                )
                self.ema[stage + "_y"] = loss_y.detach()

            if self.hparams.y_weight > 0:
                self.losses[stage + "_y"].append(loss_y.detach())

        # total loss
        loss = loss_y * self.hparams.y_weight + loss_neg_dy * self.hparams.neg_dy_weight

        self.losses[stage].append(loss.detach())
        return loss

## Método `optimizer_step`

Definimos el método `optimizer_step`, que se encarga de realizar un paso de optimización durante el entrenamiento del modelo. Aquí está lo que hace:

- **Argumentos**:
  - `*args`, `**kwargs`: Argumentos posicionales y de palabras clave que pueden ser pasados al método. En particular, esperamos encontrar el optimizador en `kwargs["optimizer"]` o en `args[2]`.
- **Procedimiento**:
  - **Escalado de tasa de aprendizaje durante el calentamiento**: Si el paso global del entrenamiento es menor que el número de pasos de calentamiento especificado en los hiperparámetros (`lr_warmup_steps`), escalamos la tasa de aprendizaje del optimizador linealmente desde 0 hasta la tasa de aprendizaje especificada. Esto se hace para suavizar el proceso de inicio del entrenamiento y evitar grandes actualizaciones de los pesos del modelo al principio.
  - **Actualización del optimizador**: Llamamos al método `optimizer_step` de la clase base para realizar la actualización real de los parámetros del modelo utilizando el optimizador proporcionado.
  - **Reinicio de los gradientes**: Después de realizar la actualización de los parámetros del modelo, reiniciamos los gradientes del optimizador utilizando `optimizer.zero_grad()` para prepararlos para el próximo paso de optimización.

Este método encapsula el proceso de optimización, incluyendo el escalado de la tasa de aprendizaje durante el calentamiento y la actualización de los parámetros del modelo.


In [ ]:
    def optimizer_step(self, *args, **kwargs):
        optimizer = kwargs["optimizer"] if "optimizer" in kwargs else args[2]
        if self.trainer.global_step < self.hparams.lr_warmup_steps:
            lr_scale = min(
                1.0,
                float(self.trainer.global_step + 1)
                / float(self.hparams.lr_warmup_steps),
            )

            for pg in optimizer.param_groups:
                pg["lr"] = lr_scale * self.hparams.lr
        super().optimizer_step(*args, **kwargs)
        optimizer.zero_grad()

## Método `training_epoch_end`

Definimos el método `training_epoch_end`, que se ejecuta al final de cada época de entrenamiento. Aquí está lo que hace:

- **Argumentos**:
  - `training_step_outputs`: Una lista de los resultados de los pasos de entrenamiento durante la época.
- **Procedimiento**:
  - **Obtención del datamodule**: Recuperamos el datamodule del entrenador (`self.trainer.datamodule`).
  - **Comprobación de la existencia de un conjunto de datos de prueba**: Verificamos si el datamodule tiene un atributo `test_dataset` y si contiene datos.
  - **Reinicio del dataloader de validación**: Si se cumplen ciertas condiciones, reiniciamos el dataloader de validación para el módulo actual. Las condiciones incluyen si el número de época actual es divisible por el intervalo de prueba especificado en los hiperparámetros (`test_interval`) o si el siguiente número de época también es divisible por este intervalo. El reinicio del dataloader de validación se realiza para preparar el dataloader para la evaluación del conjunto de datos de prueba, lo que puede ser más rápido que omitir los pasos de evaluación de prueba devolviendo `None`.

Este método permite la configuración necesaria antes y después de la evaluación del conjunto de datos de prueba al final de cada época de entrenamiento.


In [ ]:
    def training_epoch_end(self, training_step_outputs):
        dm = self.trainer.datamodule
        if hasattr(dm, "test_dataset") and len(dm.test_dataset) > 0:
            should_reset = (
                self.current_epoch % self.hparams.test_interval == 0
                or (self.current_epoch + 1) % self.hparams.test_interval == 0
            )
            if should_reset:
                # reset validation dataloaders before and after testing epoch, which is faster
                # than skipping test validation steps by returning None
                self.trainer.reset_val_dataloader(self)

## Método `validation_epoch_end`

Definimos el método `validation_epoch_end`, que se ejecuta al final de cada época de validación. Aquí está lo que hace:

- **Argumentos**:
  - `validation_step_outputs`: Una lista de los resultados de los pasos de validación durante la época.
- **Procedimiento**:
  - **Verificación del modo de chequeo de cordura**: Verificamos si el entrenamiento está en modo de chequeo de cordura (`sanity_checking`). Si no lo está, continuamos con el proceso de registro de métricas.
  - **Construcción del diccionario de métricas registradas**: Creamos un diccionario (`result_dict`) que contiene métricas importantes para el entrenamiento y la validación, como el número de época actual, la tasa de aprendizaje, la pérdida promedio de entrenamiento (`train_loss`) y la pérdida promedio de validación (`val_loss`). Si hay pérdidas para el conjunto de datos de prueba (`test_loss`), también se agregan al diccionario.
  - **Registro de las métricas**: Registramos las métricas utilizando el método `log_dict`, que sincroniza la información en todos los dispositivos en caso de entrenamiento distribuido.
  - **Reinicio de las pérdidas registradas**: Después de registrar las métricas, reiniciamos las listas de pérdidas para la próxima época.

Este método permite registrar y registrar las métricas importantes al final de cada época de validación, lo que ayuda a monitorear el rendimiento del modelo durante el entrenamiento.


In [ ]:
    def validation_epoch_end(self, validation_step_outputs):
        if not self.trainer.sanity_checking:
            # construct dict of logged metrics
            result_dict = {
                "epoch": float(self.current_epoch),
                "lr": self.trainer.optimizers[0].param_groups[0]["lr"],
                "train_loss": torch.stack(self.losses["train"]).mean(),
                "val_loss": torch.stack(self.losses["val"]).mean(),
            }

            # add test loss if available
            if len(self.losses["test"]) > 0:
                result_dict["test_loss"] = torch.stack(self.losses["test"]).mean()

            # if prediction and derivative are present, also log them separately
            if len(self.losses["train_y"]) > 0 and len(self.losses["train_neg_dy"]) > 0:
                result_dict["train_loss_y"] = torch.stack(self.losses["train_y"]).mean()
                result_dict["train_loss_neg_dy"] = torch.stack(
                    self.losses["train_neg_dy"]
                ).mean()
                result_dict["val_loss_y"] = torch.stack(self.losses["val_y"]).mean()
                result_dict["val_loss_neg_dy"] = torch.stack(self.losses["val_neg_dy"]).mean()

                if len(self.losses["test"]) > 0:
                    result_dict["test_loss_y"] = torch.stack(
                        self.losses["test_y"]
                    ).mean()
                    result_dict["test_loss_neg_dy"] = torch.stack(
                        self.losses["test_neg_dy"]
                    ).mean()

            self.log_dict(result_dict, sync_dist=True)
        self._reset_losses_dict()

## Método `_reset_losses_dict`

Definimos el método `_reset_losses_dict`, que se encarga de reiniciar el diccionario de pérdidas (`self.losses`). Aquí está lo que hace:

- **Procedimiento**:
  - **Reinicio del diccionario de pérdidas**: Creamos un nuevo diccionario llamado `self.losses` que contiene listas vacías para diferentes tipos de pérdidas, como pérdidas de entrenamiento (`train`), pérdidas de validación (`val`), pérdidas de prueba (`test`), pérdidas de predicción (`train_y`, `val_y`, `test_y`) y pérdidas de derivada negativa (`train_neg_dy`, `val_neg_dy`, `test_neg_dy`).

Este método se utiliza para reiniciar el registro de las pérdidas al comienzo de cada época, lo que garantiza que las pérdidas de la época anterior no se acumulen en las métricas registradas.


In [ ]:
    def _reset_losses_dict(self):
        self.losses = {
            "train": [],
            "val": [],
            "test": [],
            "train_y": [],
            "val_y": [],
            "test_y": [],
            "train_neg_dy": [],
            "val_neg_dy": [],
            "test_neg_dy": [],
        }

## Método `_reset_ema_dict`

Definimos el método `_reset_ema_dict`, que se encarga de reiniciar el diccionario de suavizado exponencial móvil (EMA, por sus siglas en inglés). Aquí está lo que hace:

- **Procedimiento**:
  - **Reinicio del diccionario de EMA**: Creamos un nuevo diccionario llamado `self.ema` que contiene valores `None` para diferentes tipos de EMA, como EMA de pérdidas de predicción durante el entrenamiento (`train_y`), EMA de pérdidas de predicción durante la validación (`val_y`), EMA de pérdidas de derivada negativa durante el entrenamiento (`train_neg_dy`) y EMA de pérdidas de derivada negativa durante la validación (`val_neg_dy`).

Este método se utiliza para reiniciar el seguimiento del suavizado exponencial móvil al comienzo de cada entrenamiento y validación, lo que garantiza que los valores de EMA se calculen correctamente para cada época.


In [ ]:
    def _reset_ema_dict(self):
        self.ema = {"train_y": None, "val_y": None, "train_neg_dy": None, "val_neg_dy": None}

# Ejemplo de implementación del modelo

In [ ]:
# Definir hiperparámetros
hparams = {
    "lr": 0.001,
    "weight_decay": 0.0001,
    "lr_factor": 0.1,
    "lr_patience": 10,
    "lr_min": 1e-6,
    "charge": False,
    "spin": False,
    "derivative": True,
    "ema_alpha_neg_dy": 0.99,
    "ema_alpha_y": 0.99,
    "neg_dy_weight": 0.5,
    "y_weight": 0.5,
    "lr_warmup_steps": 1000,
    "test_interval": 5
}

# Crear instancia de la clase LNNP
model = LNNP(hparams)

# Cargar conjunto de datos (MNIST como ejemplo)
train_dataset = MNIST(root='data/', train=True, download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Crear logger para visualizar en TensorBoard
logger = TensorBoardLogger("logs", name="LNNP")

# Entrenar el modelo
trainer = Trainer(logger=logger, max_epochs=10)
trainer.fit(model, train_loader)